In [1]:
import os 
import pandas as pd 
import matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime
from datetime import timedelta
import re
import numpy as np
pd.options.plotting.backend = "plotly"

In [20]:
def find_latest(RESU_dir):
    date = []
    filenames = []
    dico = {}
    for filename in os.scandir(RESU_dir):
        x = re.search("^[0-9]{8}-[0-9]{4}",filename.name)
        if x:
            y = re.search("^[0-9]{8}-[0-9]{4}_[0-9]",filename.name)
            if y:
                dt = datetime.strptime(filename.name[:13], "%Y%m%d-%H%M")
                dt += timedelta(0,60)
                new_name = dt.strftime("%Y%m%d-%H%M")
                os.rename(RESU_dir.joinpath(filename),RESU_dir.joinpath(new_name))
                fl = new_name
            else:
                dt = datetime.strptime(filename.name[:13], "%Y%m%d-%H%M")
                fl = filename.name
            filenames.append(fl)
            date.append(dt)
            dico[dt] = fl

    latest = dico[max(dico.keys())]
    last_res = RESU_dir.joinpath(latest)
    return last_res


def preprocess_listing(RESU_dir,output_name='output.txt'):
    res_dir = find_latest(RESU_dir)
    try: 
        os.mkdir(res_dir.joinpath('Graphs')) 
    except OSError as error: 
        print('Folder exists, continue')
    f = open(res_dir.joinpath("listing"),'r')
    res_dir = res_dir.joinpath('Graphs')  
    f2 = open(res_dir.joinpath(output_name),'w')
    lines = f.readlines()
    start = False
    for l in lines:
        a = re.search("v  |c  |INSTANT",l)
        if not start:
            if re.search('INSTANT',l):
                start = True
        if a and start:
            f2.write(l)
    f.close()
    f2.close()
    return res_dir.joinpath(output_name)

def extract_variable(var,filepath):
    f3 = open(filepath,'r')
    lines = f3.readlines()
    ts , content = [],[]
    for l in lines:
        a = re.search("INSTANT.*NUMBER *([0-9]*)",l)
        if a:
            ts.append(int(a.group(1)))
        b = re.search(var+' *([0-9\.eE\+-]*) *([0-9\.eE\+-]*) *([0-9\.eE\+-]*) *([0-9\.eE\+-]*) *([0-9\.eE\+-]*)',l)
        if b:
            data = []
            for g in b.groups():
                if g != '':
                    data.append(float(g))
            content.append(data)
    assert(len(ts) == len(content))
    return ts,content

def extract_fields(filepath, variables, fields):
    '''
    Extract the fields value of the specified variables
    Variables are expected to be formated like this : 
    c  var_name
    v  var_name 
    '''
    df_exist = False
    for i in range(len(variables)):
        variable = variables[i]
        col_names = []
        res = re.search('(c|v)  (\S*)',variable)
        if res:
            var_name = res.group(2)
            for var in fields:
                col_names.append((var_name,var))
            ts,content = extract_variable(variable,filepath)
            if not df_exist:
                df = pd.DataFrame(content,index=pd.Index(ts,name="Timestep"))
                df.columns = pd.MultiIndex.from_tuples(col_names)
                df_exist = True
            else:
                df1 = pd.DataFrame(content,index=pd.Index(ts,name="Timestep"))
                df1.columns = pd.MultiIndex.from_tuples(col_names)
                df = df.join(df1)
    if df_exist:
        df = df.swaplevel(axis=1)
        df = df.sort_index(axis=1)
        return df
    else:
        return None

def process_listing(res_directory):
    temporary_file = 'output.txt'
    output_path = preprocess_listing(res_directory,temporary_file)
    resu_path = output_path.parents[0]
    value_fields = ['minimum','maximum','set_mean','spatial_mean']
    value_var = ['v  ǁVelocityǁ','v  Pressure','v  k','v  epsilon','v  CourantNb']
    conv_fields = ['Rhs norm','N_iter','Norm. residual','Drift','Time residual']
    conv_var = ['c  Velocity','c  Pressure','c  k','c  epsilon']
    df_v = extract_fields(output_path, value_var, value_fields)
    df_c = extract_fields(output_path, conv_var, conv_fields)
    
    with open(resu_path.joinpath('convergence_graph.html'), 'w') as f:
        for field in conv_fields:
            source_df = df_c[field].copy(deep=True)
            fig = source_df.plot(labels={"value": field+" value"})
            fig.update_layout(
                title={'text':field, 'font':dict(size=30), 'y':0.95,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'}
            )
            f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))

    with open(resu_path.joinpath('value_graph.html'), 'w') as f:
        for field in value_fields:
            source_df = df_v[field].copy(deep=True)
            fig = source_df.plot(labels={"value": field+" value"})
            fig.update_layout(
                title={'text':field, 'font':dict(size=30), 'y':0.95,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'}
            )
            f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))

In [22]:
Doc_dir = Path("/home/juleslecoustre/Documents")
Study_name = "STUDY"
Case_Name = "FLOW"

Case_dir = Doc_dir.joinpath(Study_name).joinpath(Case_Name)
Res_dir = Case_dir.joinpath("RESU")
process_listing(Res_dir)

Folder exists, continue
